# MNIST Estimator

The original is too big to load in. Strange. This one will have a lot less comments

In [ ]:
# Import Dependancies
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import time
import numpy as np

In [ ]:
# Import Data
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# configure tf logging
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
# Declare variables
data_dir = '/tmp/MNIST_data'
model_dir_root = '/tmp/mnist_estimators'
model_dir = os.path.join(model_dir_root, "deep_" + str(int(time.time())))

# size of each batch of training examples to learn
batch_size = 100

num_steps = 15000

global DATA_SETS
print("downloading and reading data sets...")
DATA_SETS = input_data.read_data_sets(data_dir)

In [ ]:
def generate_input_fn(dataset, batch_size=batch_size):
    def _input_fn():
        X = tf.constant(dataset.images)
        Y = tf.constant(dataset.labels, dtype=tf.int32)
        image_batch, label_batch = tf.train.shuffle_batch(
            [X,Y],
            batch_size=batch_size,
            capacity=8*batch_size,
            min_after_dequeue=4*batch_size,
            enqueue_many=True
        )
        return {'pixels': image_batch}, label_batch
    return _input_fn

In [ ]:
def define_and_run_dnn_classifier(num_steps, logdir, lr=.1, batch_size=40):
    """Run a DNN classifier"""
    
    feature_columns = [tf.contrib.layers.real_valued_column("pixels", dimension=784)]
    
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=10,
        hidden_units=[128, 64, 32],
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
    )
    
    classifier.train(
        input_fn=generate_input_fn(
            DATA_SETS.train,
            batch_size=batch_size
        ),
        steps=num_steps
    )
    
    print("Finished running the deep training")
    print("evaluating DNN classifier accuracy")
    
    accuracy_score = classifier.evaluate(
        input_fn=generate_input_fn(
            DATA_SETS.test,
            batch_size=batch_size
        ),
        steps=100
    )['accuracy']
    
    print("DNN classifier accuracy: {0:f}".format(accuracy_score))

In [ ]:
print("Running DNN Classifier")
define_and_run_dnn_classifier(
    num_steps,
    model_dir
)

In [ ]:
# tensorboard --logdir=/tmp/mnist_estimators